In [1]:
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd
import numpy as np

In [2]:
def getResults():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT r.resultid, sf.samplingfeatureid, r.resultdatetime, r.featureactionid, rv.valueid, 
                                       rv.datavalue, rv.valuedatetime, sf.samplingfeaturecode,
                                       ds.datasetcode
                       FROM results r 
                       INNER JOIN measurement_result_values rv ON r.resultid=rv.resultid
                       INNER JOIN feature_actions fa ON r.featureactionid=fa.featureactionid 
                       INNER JOIN dataset_results dr ON r.resultid=dr.resultid
                       INNER JOIN datasets ds ON dr.datasetid=ds.datasetid
                       INNER JOIN sampling_features sf ON fa.samplingfeatureid=sf.samplingfeatureid
                       WHERE ds.datasetcode LIKE 'T1Data_%'""")
        rows = cur.fetchall()
        #WHERE sf.samplingfeaturecode LIKE 'NC%' ORDER BY featureactionid
        #WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'
        #WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'
        df = pd.DataFrame(columns = ['resultid', 'samplingfeatureid', 'resultdatetime', 'featureactionid', 
                                     'valueid', 'datavalue', 'valuedatetime', 'samplingfeaturecode', 
                                     'datasetcode']) 
        
        for row in rows:
            df = df.append({'resultid': row[0], 'samplingfeatureid': row[1], 'resultdatetime': row[2], 
                            'featureactionid': row[3], 'valueid': row[4], 'datavalue': row[5], 
                            'valuedatetime': row[6], 'samplingfeaturecode': row[7], 'datasetcode': row[8]}, 
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [4]:
dfdb = getResults()
dfdb.head()

,resultid,samplingfeatureid,resultdatetime,featureactionid,valueid,datavalue,valuedatetime,samplingfeaturecode,datasetcode
0,61338,21196,2019-07-03 16:00:00+00:00,3653,51306,1.556,2019-07-03 16:00:00+00:00,NC0195118_Specimen,T1Data_Knappe_NC0195118_Specimen
1,61339,21196,2019-07-03 16:00:00+00:00,3653,51307,6.010,2019-07-03 16:00:00+00:00,NC0195118_Specimen,T1Data_Knappe_NC0195118_Specimen
2,61340,21196,2019-07-03 16:00:00+00:00,3653,51308,2.072,2019-07-03 16:00:00+00:00,NC0195118_Specimen,T1Data_Knappe_NC0195118_Specimen
3,61341,21196,2019-07-03 16:00:00+00:00,3653,51309,2.153,2019-07-03 16:00:00+00:00,NC0195118_Specimen,T1Data_Knappe_NC0195118_Specimen
4,61342,21416,2019-08-12 16:00:00+00:00,3655,51310,7.100,2019-08-12 16:00:00+00:00,NC0188115_Specimen,T1Data_Ferguson_NC0188115_Specimen
...,...,...,...,...,...,...,...,...,...
810,62148,21533,2019-08-22 16:00:00+00:00,4011,52116,6.224,2019-08-22 16:00:00+00:00,NC5009012_Specimen,T1Data_Knappe_NC5009012_Specimen
811,62149,21533,2019-08-22 16:00:00+00:00,4011,52117,7.737,2019-08-22 16:00:00+00:00,NC5009012_Specimen,T1Data_Knappe_NC5009012_Specimen
812,62150,21533,2019-08-22 16:00:00+00:00,4011,52118,33.159,2019-08-22 16:00:00+00:00,NC5009012_Specimen,T1Data_Knappe_NC5009012_Specimen
813,62151,21477,2019-08-24 16:00:00+00:00,4013,52119,2.816,2019-08-24 16:00:00+00:00,NC0363010_Specimen,T1Data_Knappe_NC0363010_Specimen


In [4]:
len(dfdb['resultid'])

815

In [5]:
def getResAllBridgesView():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT res.resultid, fa.samplingfeatureid, rf.to_samplingfeature_id, dsr.datasetid, 
                              ab.actionid, ab.affiliationid, dsc.citationid, dsc.relationshiptypecv
                       FROM results res
                           LEFT JOIN feature_actions fa ON res.featureactionid = fa.featureactionid
                           LEFT JOIN action_by ab ON fa.actionid = ab.actionid
                           LEFT JOIN dataset_results dsr ON dsr.resultid = res.resultid
                           LEFT JOIN dataset_citations dsc ON dsc.datasetid = dsr.datasetid
                           LEFT JOIN related_features rf ON rf.from_samplingfeature_id = fa.samplingfeatureid
                           AND rf.relationshiptypecv::text = 'Was collected at'::text
                           WHERE fa.samplingfeatureid = 19665
                       ORDER BY fa.samplingfeatureid""")
        #WHERE fa.samplingfeatureid = 21196
        rows = cur.fetchall()
        df = pd.DataFrame(columns = ['resultid','samplingfeatureid','to_samplingfeature_id','datasetid', 
                                     'actionid','affiliationid','citationid','relationshiptypecv']) 
        
        for row in rows:
            df = df.append({'resultid':row[0],'samplingfeatureid':row[1],'to_samplingfeature_id':row[2],
                            'datasetid':row[3],'actionid':row[4],'affiliationid':row[5],'citationid':row[6],
                            'relationshiptypecv':row[7]}, 
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()        

In [6]:
df = getResAllBridgesView()
df.head()

,resultid,samplingfeatureid,to_samplingfeature_id,datasetid,actionid,affiliationid,citationid,relationshiptypecv
0,44465.0,19665.0,19285.0,2301.0,2764.0,10005.0,NaN,NaN
1,44464.0,19665.0,19285.0,2301.0,2764.0,10005.0,NaN,NaN
2,44463.0,19665.0,19285.0,2301.0,2764.0,10005.0,NaN,NaN
3,44462.0,19665.0,19285.0,2301.0,2764.0,10005.0,NaN,NaN


In [24]:
#outfile = '../resultsT2Data.csv'
#dfdb.to_csv(outfile, header=True, index=False)

In [12]:
len(np.unique(np.array(dfdb['datasetcode'])))

181

In [13]:
len(dfdb['datasetcode'])

815

In [10]:
len(np.unique(np.array(dfdb['resultid'])))

815

In [11]:
len(dfdb['resultid'])

815

In [16]:
def getActions():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT DISTINCT r.resultid, r.resultdatetime, rv.valueid, rv.datavalue, rv.valuedatetime,
                                       a.actionid, a.begin_datetime, a.end_datetime, a.methodid
                       FROM results r 
                       INNER JOIN measurement_result_values rv ON r.resultid=rv.resultid
                       INNER JOIN feature_actions fa ON r.featureactionid=fa.featureactionid
                       INNER JOIN actions a ON fa.actionid=a.actionid
                       INNER JOIN sampling_features sf ON fa.samplingfeatureid=sf.samplingfeatureid
                       WHERE sf.samplingfeaturecode LIKE 'NC%'""")
        rows = cur.fetchall()
        #WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'
        
        df = pd.DataFrame(columns = ['resultid', 'resultdatetime', 'valueid', 'datavalue', 'valuedatetime',
                                     'actionid', 'begin_datetime', 'end_datetime', 'methodid']) 
        
        for row in rows:
            df = df.append({'resultid': row[0], 'resultdatetime': row[1], 'valueid': row[2], 
                            'datavalue': row[3], 'valuedatetime': row[4], 'actionid': row[5], 
                            'begin_datetime': row[6], 'end_datetime': row[7], 'methodid': row[8]}, 
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [17]:
dfdb = getActions()
dfdb.head()

,resultid,resultdatetime,valueid,datavalue,valuedatetime,actionid,begin_datetime,end_datetime,methodid
0,58893,2019-07-03 16:00:00+00:00,48861,1.556,2019-07-03 16:00:00+00:00,2884,2019-07-03 00:00:00+00:00,2019-07-03 23:59:59+00:00,10003
1,58894,2019-07-03 16:00:00+00:00,48862,6.010,2019-07-03 16:00:00+00:00,2884,2019-07-03 00:00:00+00:00,2019-07-03 23:59:59+00:00,10003
2,58895,2019-07-03 16:00:00+00:00,48863,2.072,2019-07-03 16:00:00+00:00,2884,2019-07-03 00:00:00+00:00,2019-07-03 23:59:59+00:00,10003
3,58896,2019-07-03 16:00:00+00:00,48864,2.153,2019-07-03 16:00:00+00:00,2884,2019-07-03 00:00:00+00:00,2019-07-03 23:59:59+00:00,10003
4,58897,2019-08-12 16:00:00+00:00,48865,7.100,2019-08-12 16:00:00+00:00,2885,2019-08-12 00:00:00+00:00,2019-08-12 23:59:59+00:00,10005


In [10]:
len(np.unique(np.array(dfdb['actionid'])))

380

In [11]:
len(dfdb['actionid'])

18171

In [12]:
len(np.unique(np.array(dfdb['end_datetime'])))

44

In [13]:
def getFeatures():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT DISTINCT r.resultid, r.resultdatetime, rv.valueid, rv.datavalue, rv.valuedatetime,
                                       a.actionid, a.begin_datetime, a.end_datetime, a.methodid,
                                        sf.samplingfeatureid, ST_X(ST_AsText(sf.featuregeometry))
                       FROM results r 
                       INNER JOIN measurement_result_values rv ON r.resultid=rv.resultid
                       INNER JOIN feature_actions fa ON r.featureactionid=fa.featureactionid
                       INNER JOIN actions a ON fa.actionid=a.actionid
                       INNER JOIN sampling_features sf ON fa.samplingfeatureid=sf.samplingfeatureid
                       WHERE sf.samplingfeaturecode LIKE 'NC%'""")
        rows = cur.fetchall()
        #WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'
        
        df = pd.DataFrame(columns = ['resultid', 'resultdatetime', 'valueid', 'datavalue', 'valuedatetime',
                                     'actionid', 'begin_datetime', 'end_datetime', 'methodid',
                                      'samplingfeatureid', 'lon']) 
        
        for row in rows:
            df = df.append({'resultid': row[0], 'resultdatetime': row[1], 'valueid': row[2], 
                            'datavalue': row[3], 'valuedatetime': row[4], 'actionid': row[5], 
                            'begin_datetime': row[6], 'end_datetime': row[7], 'methodid': row[8],
                            'samplingfeatureid': row[9], 'lon': row[10]},
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [14]:
dfdb = getFeatures()
dfdb.head()

,resultid,resultdatetime,valueid,datavalue,valuedatetime,actionid,begin_datetime,end_datetime,methodid,samplingfeatureid,lon
0,10919,2019-07-02 12:00:00-04:00,891,1.0,2019-07-02 12:00:00-04:00,196,2019-07-02 00:00:00-04:00,2019-07-02 23:59:59-04:00,10003,18882,-81.134111
1,10920,2019-07-02 12:00:00-04:00,892,5.0,2019-07-02 12:00:00-04:00,196,2019-07-02 00:00:00-04:00,2019-07-02 23:59:59-04:00,10003,18882,-81.134111
2,10921,2019-07-02 12:00:00-04:00,893,10.0,2019-07-02 12:00:00-04:00,196,2019-07-02 00:00:00-04:00,2019-07-02 23:59:59-04:00,10003,18882,-81.134111
3,10922,2019-07-02 12:00:00-04:00,894,2.0,2019-07-02 12:00:00-04:00,196,2019-07-02 00:00:00-04:00,2019-07-02 23:59:59-04:00,10003,18882,-81.134111
4,10923,2019-07-02 12:00:00-04:00,895,2.0,2019-07-02 12:00:00-04:00,196,2019-07-02 00:00:00-04:00,2019-07-02 23:59:59-04:00,10003,18882,-81.134111


In [15]:
np.array(dfdb['lon'])

array([-79.47892666, -78.02686657, -76.35896649, ..., -77.30003197,
       -82.22916199, -78.74967816])

In [16]:
len(np.unique(np.array(dfdb['samplingfeatureid'])))

380